# Components

### Packages & libraries

In [1]:
import numpy as np
import pandas as pd
import os
import datetime as dt
import json
import requests
from bs4 import BeautifulSoup

### Global variables

In [41]:
file_flights = 'data/flights_kabfusab1b231239bdahb312421.xlsx'
path_passengers = 'data/passengers/'
path_output = 'output/'
sheet_flights = 'pax_per_flight'
sheet_pax = 'pax_data'
sheet_log = 'warnings-errors'
url_iata = 'https://nl.wikipedia.org/wiki/Vliegvelden_gesorteerd_naar_IATA-code'

### Functions

In [3]:
def save_to_excel(filename, dfs):
    """
    Given a dict of dataframes, for example:
    dfs = {'gadgets': df_gadgets, 'widgets': df_widgets}
    
    Function courtesy of T. Hellemans, course Data Science, Syntra, 2022-23
    """

    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    for sheetname, df in dfs.items():  # loop through `dict` of dataframes
        df.to_excel(writer, sheet_name=sheetname, index=False)  # send df to writer
        worksheet = writer.sheets[sheetname]  # pull worksheet object
        for idx, col in enumerate(df):  # loop through all columns
            series = df[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 1  # adding a little extra space
            worksheet.set_column(idx, idx, max_len)  # set column width
    writer.save()
    return


In [4]:
def build_datetime(date, time):
    """
    Parameters:
    - date: string with date in format DD/MM/YYYY
    - time: string with time in format HH:MM
    
    Result:
    - datetime object
    """

    year = pd.to_datetime(date, format='%d/%m/%Y').year
    month = pd.to_datetime(date, format='%d/%m/%Y').month
    day = pd.to_datetime(date, format='%d/%m/%Y').day
    hours = pd.to_datetime(time, format='%H:%M').hour
    minutes = pd.to_datetime(time, format='%H:%M').minute
    
    string = str(year) + '-' + str(month) + '-' + str(day) + '-' + str(hours) + '-' + str(minutes)
    
    return(pd.to_datetime(string, format='%Y-%m-%d-%H-%M'))

In [5]:
def iata_codes_from_wikipedia(url):
    """
    Parameters:
    - URL to data source (nl.wikipedia.org)
    
    Result:
    - dataframe with IATA-codes and location information
    """
    column_headers = []
    data = []
    idx = []

# Get data from website
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    html_table = soup.find('table', attrs={'class': 'wikitable sortable'})
    html_table_body = html_table.find('tbody') # tbody = table body
    header = html_table_body.find_all('th')    # th = table header
    table = html_table_body.find_all('tr')     # tr = table row

# Extract column headers
    for element in header:
        column_headers.append(element.text.strip())   # 'strip' to remove a.o. line feed

# Extract data
    for row in table:
        data_line = {}
        line = row.find_all('td')              # td = table data
        if len(line) > 0:                      # table header is also a 'tr' line but without 'td'
            for i, element in enumerate(line):
                if i == 0:
                    idx.append(element.text.strip())
                else:
                    # Sometimes apparently there is also a text for the flag shown
                    # --> only keep text for country
                    data_line[column_headers[i]] = element.text.strip().split('\xa0')[-1]   
            data.append(data_line)

# Build dataframe
    df = pd.DataFrame(data, index = idx)
    return(df)

In [6]:
def map_iata(airport, df):
    return(df.loc[airport]['Land'])

# Main program

### Read flight data

In [7]:
df_flights = pd.read_excel(file_flights)
df_flights['sched_datetime'] = df_flights.apply(lambda x: build_datetime(x['date'], x['time']), axis=1)
display(df_flights.head(5))
df_flights.dtypes

,code,A/D,airline,date,time,airport,sched_datetime
0,FLIGHT_90013,A,BA,20/12/2022,06:12,BRU,2022-12-20 06:12:00
1,FLIGHT_18336,D,RN,20/12/2022,06:12,BRU,2022-12-20 06:12:00
2,FLIGHT_15001,D,IT,20/12/2022,06:12,AMS,2022-12-20 06:12:00
3,FLIGHT_76846,D,RN,20/12/2022,06:12,AMD,2022-12-20 06:12:00
4,FLIGHT_99980,A,IT,20/12/2022,06:12,BRU,2022-12-20 06:12:00


code                      object
A/D                       object
airline                   object
date                      object
time                      object
airport                   object
sched_datetime    datetime64[ns]
dtype: object

### Read passenger data & build worksheet <em>pax_data</em>

<em>Vraag: Kan dit via UFunc ?</em>

In [24]:
content = []
log = []

for index, row in df_flights.iterrows():
    lv_code = row['code'][-5:]
    
    if lv_code.isdigit():
        
        lv_path = path_passengers + lv_code + '.json'
    
        try:
            with open(lv_path, 'r') as pax_file:
                pax_list = json.load(pax_file)
        except Exception as error:
            log_line = ' '.join([f"No passenger file found for flight {row['code'][-5:]} of airline {row['airline']}",
                                 f"{'arriving from' if row['A/D'] == 'A' else 'departing to'} {row['airport']}",
                                 f"on {row['sched_datetime'].strftime(format='%Y/%m/%d %H:%M')}."])
            log.append({'Note' : log_line})
            
        for pax_row in pax_list:
            row2 = {'FlightCode' : row['code'], \
                    'FirstName' : pax_row['first_name'], 'LastName' : pax_row['last_name'], \
                    'Airline' : row['airline'], 'A/D' : row['A/D'], 'Airport' : row['airport'], \
                    'PRM' : pax_row['needs_assistance'], 'sched_datetime' : row['sched_datetime'] }
            content.append(row2)
    
    else:
        # Invalid flight no.
        log_line = ' '.join([f"Invalid flight no. for flight on line {index + 2} of airline {row['airline']}",
                             f"{'arriving from' if row['A/D'] == 'A' else 'departing to'}  {row['airport']}",
                             f"on {row['sched_datetime'].strftime(format='%Y/%m/%d %H:%M')}."])
        log.append({'Note' : log_line})
        
df_pax = pd.DataFrame(content)
df_pax['PRM'] = df_pax['PRM'].astype(bool)
display(df_pax.head(5))
display(log)

,FlightCode,FirstName,LastName,Airline,A/D,Airport,PRM,sched_datetime
0,FLIGHT_90013,7216034,3582476,BA,A,BRU,False,2022-12-20 06:12:00
1,FLIGHT_90013,8778140,5559138,BA,A,BRU,False,2022-12-20 06:12:00
2,FLIGHT_90013,7062740,6984582,BA,A,BRU,False,2022-12-20 06:12:00
3,FLIGHT_90013,9152501,4680251,BA,A,BRU,False,2022-12-20 06:12:00
4,FLIGHT_90013,6378899,3667599,BA,A,BRU,False,2022-12-20 06:12:00


[{'Note': 'No passenger file found for flight 15001 of airline IT departing to AMS on 2022/12/20 06:12.'},
 {'Note': 'Invalid flight no. for flight on line 29 of airline SA departing to  AMS on 2022/12/20 07:12.'}]

In [26]:
# Reverse check: Do the flights of all passenger files exist in the main file ?
for flightno in os.listdir(path='data/passengers/'):
    if str('FLIGHT_' + flightno[0:5]) not in df_flights['code'].unique():
        log_line = f"No flight data found for flight {flightno[0:5]}."
        log.append({'Note' : f"No flight data found for flight {flightno[0:5]}."})

display(log)

No flight data found for flight 72276.


[{'Note': 'No passenger file found for flight 15001 of airline IT departing to AMS on 2022/12/20 06:12.'},
 {'Note': 'Invalid flight no. for flight on line 29 of airline SA departing to  AMS on 2022/12/20 07:12.'},
 {'Note': 'No flight data found for flight 72276.'}]

### Build worksheet <em>pax_per_flight</em>

In [29]:
df_count = df_pax.groupby('FlightCode').agg({'PRM' : 'count'})
df_count.rename({'PRM' : 'pax'},axis=1, inplace=True)

df_prm = df_pax.groupby(['FlightCode', 'sched_datetime', 'Airline', 'A/D', 'Airport'], as_index=False).agg({'PRM' : 'sum'})

df_ppf = pd.merge(df_prm, df_count, on='FlightCode')

df_ppf.drop('FlightCode', axis=1, inplace=True)
display(df_ppf.head(5))
df_ppf.dtypes

,sched_datetime,Airline,A/D,Airport,PRM,pax
0,2022-12-20 17:12:00,AF,D,LGW,6,269
1,2022-12-20 19:12:00,SN,D,AMD,6,294
2,2022-12-20 21:12:00,RN,A,AMS,5,152
3,2022-12-20 07:12:00,SN,D,AMD,2,130
4,2022-12-20 16:12:00,BA,A,BRU,4,251


sched_datetime    datetime64[ns]
Airline                   object
A/D                       object
Airport                   object
PRM                        int64
pax                        int64
dtype: object

### Complete IATA data

In [30]:
# Get additional IATA data
df_iata = iata_codes_from_wikipedia(url_iata)
display(df_iata)

,ICAO,Luchthaven,Stad,Land
AAA,NTGA,Anaa,"Anaa, Tuamotueilanden",Frans-Polynesië
AAB,YARY,Arrabury,Arrabury,Australië
AAC,HEAR,Al Arish International,Al Arish,Egypte
AAD,,Ad-Dabbah,Ad-Dabbah,Soedan
AAE,DABB,Rabah Bitat (Les Salines),Annaba,Algerije
...,...,...,...,...
ZNZ,HTZA,Zanzibar International,Zanzibar,Tanzania
ZQW,EDRZ,Zweibrücken,Saarbrücken,Duitsland
ZRH,LSZH,Zurich Airport,Zurich,Zwitserland
ZSE,FMEP,Pierrefonds,Saint Pierre de la Réunion,Frankrijk


In [31]:
df_ppf['Country'] = df_ppf.apply(lambda x: map_iata(x['Airport'], df_iata), axis=1)
display(df_ppf)

,sched_datetime,Airline,A/D,Airport,PRM,pax,Country
0,2022-12-20 17:12:00,AF,D,LGW,6,269,Verenigd Koninkrijk
1,2022-12-20 19:12:00,SN,D,AMD,6,294,India
2,2022-12-20 21:12:00,RN,A,AMS,5,152,Nederland
3,2022-12-20 07:12:00,SN,D,AMD,2,130,India
4,2022-12-20 16:12:00,BA,A,BRU,4,251,België
...,...,...,...,...,...,...,...
561,2022-12-20 10:12:00,MS,A,LGW,7,131,Verenigd Koninkrijk
562,2022-12-20 23:12:00,TG,D,AMD,7,252,India
563,2022-12-20 14:12:00,RN,A,BRU,10,312,België
564,2022-12-20 23:12:00,AF,D,LGW,4,222,Verenigd Koninkrijk


### Prep output

In [28]:
df_log = pd.DataFrame(log)
display(df_log)

,Note
0,No passenger file found for flight 15001 of ai...
1,Invalid flight no. for flight on line 29 of ai...
2,No flight data found for flight 72276.


In [36]:
# filename <- flight date
file_output = path_output + df_ppf['sched_datetime'].iloc[0].strftime('%d-%m-%Y') + '.xlsx'

# Convert log to dataframe
df_log = pd.DataFrame(log)

# Format PRM to 'Y', 'N'
df_pax['PRM'] = df_pax['PRM'].transform(lambda x: 'Y' if x == True else 'N')

# Sort worksheets
df_ppf.sort_values(by=['sched_datetime', 'Airline', 'A/D', 'Airport'], inplace=True, ignore_index=True)
df_pax.sort_values(by=['sched_datetime', 'FlightCode'], inplace=True, ignore_index=True)

display(df_log.head())
display(df_ppf.head())
display(df_pax.head())

,Note
0,No passenger file found for flight 15001 of ai...
1,Invalid flight no. for flight on line 29 of ai...
2,No flight data found for flight 72276.


,FlightCode,FirstName,LastName,Airline,A/D,Airport,PRM,sched_datetime
0,FLIGHT_12833,9311105,9994947,A2,D,BRU,N,2022-12-20 06:12:00
1,FLIGHT_12833,4664322,4932406,A2,D,BRU,N,2022-12-20 06:12:00
2,FLIGHT_12833,8376344,6894155,A2,D,BRU,N,2022-12-20 06:12:00
3,FLIGHT_12833,2418920,4090089,A2,D,BRU,N,2022-12-20 06:12:00
4,FLIGHT_12833,1156252,3948337,A2,D,BRU,N,2022-12-20 06:12:00


,sched_datetime,Airline,A/D,Airport,PRM,pax,Country
0,2022-12-20 06:12:00,A2,A,AMD,8,297,India
1,2022-12-20 06:12:00,A2,D,BRU,3,266,België
2,2022-12-20 06:12:00,AF,D,AMD,4,201,India
3,2022-12-20 06:12:00,AF,D,AMS,4,166,Nederland
4,2022-12-20 06:12:00,BA,A,AMS,2,211,Nederland


,FlightCode,FirstName,LastName,Airline,A/D,Airport,PRM,sched_datetime
0,FLIGHT_12833,9311105,9994947,A2,D,BRU,N,2022-12-20 06:12:00
1,FLIGHT_12833,4664322,4932406,A2,D,BRU,N,2022-12-20 06:12:00
2,FLIGHT_12833,8376344,6894155,A2,D,BRU,N,2022-12-20 06:12:00
3,FLIGHT_12833,2418920,4090089,A2,D,BRU,N,2022-12-20 06:12:00
4,FLIGHT_12833,1156252,3948337,A2,D,BRU,N,2022-12-20 06:12:00


In [37]:
# datetime -> string
df_ppf['ScheduledTime'] = df_ppf.apply(lambda x: x['sched_datetime'].strftime('%Y/%m/%d %H:%M'), axis=1)
df_ppf = df_ppf[['ScheduledTime', 'Country', 'Airline', 'A/D', 'Airport', 'PRM', 'pax']]

df_pax['ScheduledTime'] = df_pax.apply(lambda x: x['sched_datetime'].strftime('%Y/%m/%d %H:%M'), axis=1)
df_pax.drop('sched_datetime', axis=1, inplace=True)

In [38]:
display(df_log.head(5))
display(df_ppf.head(5))
display(df_pax.head(5))

,Note
0,No passenger file found for flight 15001 of ai...
1,Invalid flight no. for flight on line 29 of ai...
2,No flight data found for flight 72276.


,ScheduledTime,Country,Airline,A/D,Airport,PRM,pax
0,2022/12/20 06:12,India,A2,A,AMD,8,297
1,2022/12/20 06:12,België,A2,D,BRU,3,266
2,2022/12/20 06:12,India,AF,D,AMD,4,201
3,2022/12/20 06:12,Nederland,AF,D,AMS,4,166
4,2022/12/20 06:12,Nederland,BA,A,AMS,2,211


,FlightCode,FirstName,LastName,Airline,A/D,Airport,PRM,ScheduledTime
0,FLIGHT_12833,9311105,9994947,A2,D,BRU,N,2022/12/20 06:12
1,FLIGHT_12833,4664322,4932406,A2,D,BRU,N,2022/12/20 06:12
2,FLIGHT_12833,8376344,6894155,A2,D,BRU,N,2022/12/20 06:12
3,FLIGHT_12833,2418920,4090089,A2,D,BRU,N,2022/12/20 06:12
4,FLIGHT_12833,1156252,3948337,A2,D,BRU,N,2022/12/20 06:12


### Output

In [42]:
# Generate Excel file
workbook = {sheet_log : df_log, sheet_flights : df_ppf, sheet_pax : df_pax}
save_to_excel(file_output, workbook)